In [10]:
import pandas as pd
from collections import defaultdict

In [3]:
processed_bids = pd.read_pickle("../outputs/processed_bids.pkl")

,bidder,GEOID,round,all_neighbors,neighbors,winner,won_blocks,geometry,neighbor_status,num_blocks_won,neighbor_bgs,all_neighbor_bgs
0,"1stel, Inc.",10010207001,9,"{10010202002: {('WOW', 'WideOpenWest Finance, ...",[],False,[],"POLYGON ((-86.47038 32.45658, -86.47036 32.456...",0,0,0,11
1,"Hughes Network Systems, LLC",10010207001,13,"{10010202002: {('WOW', 'WideOpenWest Finance, ...",[],False,[],"POLYGON ((-86.47038 32.45658, -86.47036 32.456...",0,0,0,11
2,Simple Networks Systems LLC,10010207001,10,"{10010202002: {('WOW', 'WideOpenWest Finance, ...",[],False,[],"POLYGON ((-86.47038 32.45658, -86.47036 32.456...",0,0,0,11
3,Space Exploration Technologies Corp.,10010207001,13,"{10010202002: {('WOW', 'WideOpenWest Finance, ...",[],True,"[10010207001071, 10010207001073, 10010207001095]","POLYGON ((-86.47038 32.45658, -86.47036 32.456...",0,3,0,11
4,"Viasat, Inc.",10010207001,12,"{10010202002: {('WOW', 'WideOpenWest Finance, ...",[],False,[],"POLYGON ((-86.47038 32.45658, -86.47036 32.456...",0,0,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...
424126,"Hughes Network Systems, LLC",560430002002,12,{560039626001: {('Mountain West Technologies C...,[],False,[],"POLYGON ((-108.20175 43.82671, -108.20152 43.8...",0,0,0,10
424127,"Plains Internet, LLC",560430002002,10,{560039626001: {('Mountain West Technologies C...,[],False,[],"POLYGON ((-108.20175 43.82671, -108.20152 43.8...",0,0,0,10
424128,Simple Networks Systems LLC,560430002002,1,{560039626001: {('Mountain West Technologies C...,[],False,[],"POLYGON ((-108.20175 43.82671, -108.20152 43.8...",0,0,0,10
424129,Space Exploration Technologies Corp.,560430002002,13,{560039626001: {('Mountain West Technologies C...,[],True,"[560430002002314, 560430002002337, 56043000200...","POLYGON ((-108.20175 43.82671, -108.20152 43.8...",0,7,0,10


In [ ]:
new_table = defaultdict(dict)
for idx, row in processed_bids.iterrows():
    id = row["GEOID"]
    bidder = row["bidder"]
    if "bidders" not in new_table[id]:
        new_table[id]["bidders"] = dict()
    new_table[id]["all_neighbors"] = row["all_neighbors"]
    new_table[id]["geometry"] = row["geometry"]
    new_table[id]["bidders"][bidder] = {"neighbors": row["neighbors"], "round": row["round"],
                                        "neighbor_status": row["neighbor_status"]}
    if row["winner"]:
        new_table[id]["winner"] = bidder
        new_table[id]["blocks"] = row["won_blocks"]
        new_table[id]["round"] = row["round"]

In [50]:
collapsed_bids = pd.DataFrame.from_dict(new_table , orient='index')
collapsed_bids = collapsed_bids.reset_index().rename(columns={"index":"GEOID"})
collapsed_bids

,GEOID,bidders,all_neighbors,geometry,winner,blocks,round
0,10010207001,"{'1stel, Inc.': {'neighbors': [], 'round': 9, ...","{10010202002: {('WOW', 'WideOpenWest Finance, ...","POLYGON ((-86.47038099999999 32.456581, -86.47...",Space Exploration Technologies Corp.,"[10010207001071, 10010207001073, 10010207001095]",True
1,10010208011,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010206002: {('WOW', 'WideOpenWest Finance, ...","POLYGON ((-86.59549799999999 32.381123, -86.59...",Space Exploration Technologies Corp.,"[10010208011010, 10010208011017, 1001020801102...",True
2,10010208012,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010201001: {('WOW', 'WideOpenWest Finance, ...","POLYGON ((-86.591984 32.44164, -86.591976 32.4...",Space Exploration Technologies Corp.,"[10010208012004, 10010208012010, 1001020801201...",True
3,10010208021,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010201001: {('WOW', 'WideOpenWest Finance, ...","POLYGON ((-86.645787 32.554375, -86.6457779999...",RDOF USA Consortium,"[10010208021003, 10010208021006, 1001020802101...",True
4,10010209001,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010208021: {('Central Access, Inc.', 'Centr...","POLYGON ((-86.664761 32.627948, -86.6647069999...",RDOF USA Consortium,"[10010209001000, 10010209001001, 1001020900100...",True
...,...,...,...,...,...,...,...
61736,560399676003,{'AMG Technology Investment Group LLC': {'neig...,"{160199701001: {('Safelink Internet'), ('Alway...","POLYGON ((-111.045492 43.51067099999999, -111....",Space Exploration Technologies Corp.,"[560399676003075, 560399676003087, 56039967600...",True
61737,560399677021,{'AMG Technology Investment Group LLC': {'neig...,"{560399676001: {('Rise Broadband', 'JAB Wirele...","POLYGON ((-111.039007 43.472801, -111.036624 4...",Space Exploration Technologies Corp.,"[560399677021047, 560399677021058, 56039967702...",True
61738,560419753001,{'AMG Technology Investment Group LLC': {'neig...,"{490439641011: {('Union Holding Corp.', 'Union...","POLYGON ((-111.046818 41.156019, -111.046818 4...",Space Exploration Technologies Corp.,[560419753001107],True
61739,560430002001,{'AMG Technology Investment Group LLC': {'neig...,{560039626002: {('Mountain West Technologies C...,"POLYGON ((-108.550788 44.12770099999999, -108....",Space Exploration Technologies Corp.,"[560430002001181, 560430002001513]",True


In [30]:
bids = pd.read_csv("../data/all_bids.csv")
bidders = bids.groupby("bidder").agg({"t+l_weight":"first", "tier":"first", "latency":"first"}).reset_index()
name_map = {
    "CCO Holdings, LLC":"Charter Communications Inc",
    "Windstream Services LLC, Debtor-In-Possession":"Windstream Holdings, Inc.",
    "Connect Everyone LLC":"Starry, Inc.",
    "Frontier Communications Corporation, DIP":"Frontier Communications Corporation",
    "California Internet, L.P. dba GeoLinks":"GeoLinks",
    "Etheric Communications LLC":"Etheric Networks, Inc.",
    "Consolidated  Communications, Inc.":"Consolidated Communications, Inc.",
    "Frontier Communications Northwest, LLC":"Ziply Fiber",
    "Citynet West Virginia, LLC":"Citynet, LLC",
    "Computer 5, Inc. d/b/a LocalTel Communications": "LocalTel Communications",
    "Armstrong Telephone Company - Northern Division": "Armstrong Holdings, Inc.",
    "Commnet Wireless, LLC": "ATN International, Inc.",
    "Mediacom Communications Corporation": "Mediacom Communications Corp.",
    "Continental Divide Electric Cooperative": "Continental Divide Electric Cooperative, Inc.",
    "South Arkansas Telephone Company": "South Arkansas Telephone Co.",
    "Digital Connections Inc. dba PRODIGI": "Digital Connections, Inc.",
    "Shenandoah Cable Television, LLC": "Shenandoah Telecommunications Company",
    "Direct Communications Rockland, Inc": "Direct Communications Rockland, Inc.",
    "Pine Belt Communications, Inc.":"Pine Belt Communications Co. Inc.",
    "Savage Communications":"Savage Communications Inc.",
    "Hamilton County Telephone Co-op":"Hamilton County Telephone Co-Op",
    "Hotwire Communications, Ltd":"Hotwire Communications Ltd.",
    "Altice USA, Inc.":"Altice",
    "TruVista Communications, Inc.":"TruVista Communications of Georgia, LLC",
    "Hawaii Dialogix Telecom LLC":"Hawaii Dialogix Telecom LLC",
    "Safelink Internet LLC":"Safelink Internet",
    "American Heartland":"Farmers Telephone Company",
    "Cellular Services LLC.":"Cellular Services LLC",
    "Scott County Telephone Cooperative, Inc.":"Scott County",
    "Micrologic Inc.":"Micrologic, Inc.",
    "Pioneer Wireless, Inc":"Pioneer Wireless, Inc.",
    "St. John Telco":"St. John Telephone, Inc.",
    "One Ring Networks, Inc":"One Ring Networks, Inc.",
    "Emery Telephone dba Emery Telcom":"Emery Telcom",
    "XIT Telecommunication & Technology":"XIT Rural Telephone Cooperative, Inc.",
    "Somerset Telephone Co., Inc.":"SOMERSET TELEPHONE COMPANY",
    "Siuslaw Broadband, LLC dba Hyak Technologies":"Siuslaw Broadband, LLC",
    "Minnesota Connections c/o Consolidated Tel Company":"Consolidated Telephone Company",
    "DoCoMo Pacific, Inc.":"Docomo Pacific",
    "Roseau Electric Cooperative, Inc.":"Roseau Electric Cooperative",
    "Custer Telephone Cooperative, Inc.":"Custer Telephone Cooperative Inc.",
    "Lakeland Communications Group, LLC":"Lakeland Communications, Inc.",
    "LigTel Communications, Inc.":"Ligtel Communications",
    "Hamilton Long Distance Company":"Hamilton.net, Inc.",
    "Allen's T.V. Cable Service, Inc.":"Allen's TV Cable Service, Inc.",
    "W. T. Services, Inc.":"W.T. Services, Inc.",
    "Heart of the Catskills Comm. Inc., dba MTC Cable":"MTC Cable",
    "Citizens Vermont Acquisition Corporation":"Citizens Telephone Company",
    "Nova Cablevision, Inc":"Nova Cablevision, Inc.",
    "yondoo Broadband LLC":"yondoo Broadband, LLC",
    "Baraga Telephone Company":"Baraga Telephone Company Inc.",
    "Miles Communications LLC":"Miles Communications, Inc.",
    "PVT NetWorks, Inc.":"Penasco Valley Telephone Cooperative, Inc.",
    "Yucca Telecommunications Systems, Inc.":"Yucca Telecom",
    "H&B Communication's, Inc.":"H&B Enterprises Inc.",
    "MARQUETTE-ADAMS TELEPHONE COOPERATIVE, INC.":"Marquette-Adams Telephone Cooperative, Inc.",
    "Pinpoint Bidding Coalition":"Pinpoint Holdings, Inc.",
    "Computer Techniques, Inc. dba CTI Fiber":"CTI Fiber",
    "Barry Technology Services, LLC":"Barry Technology Services",
    "Bloosurf, LLC":"Bloosurf",
    "St Paul Cooperative Telephone Association":"St Paul Coop Telephone Assoc",
    "Farmers Mutual Cooperative Telephone Company":"FARMERS MUTUAL TELEPHONE COMPANY",
    "NTS Communications, LLC":"NTS, Inc.",
    "Unified Communications Inc.":"Unified Communications, Inc.",
    "Peoples Communication, LLC.":"Peoples Communication, Inc.",
    "Wood County Telephone Company d/b/a Solarus":"Solarus",
    "Comcell Inc.":"Comcell, Inc.",
    "Taylor Telephone Coop., Inc. dba Taylor Telecom":"Taylor Telephone Cooperative, Inc.",
    "Plateau Telecommunications, Incorporated":"Plateau",
    "AMA Communications, LLC":"AMA Communicaitons, L.L.C.",
    "AirCell, Inc.":"AirCell",
    "Bays-ET Highspeed LLC":"Bays-ET Highspeed Internet LLC",
    "Big Bend Telecom LTD":"Big Bend Telephone Company, Inc.",
    "Blue Ridge Cable Technologies, Inc.":"Blue Ridge Cable Technologies, Inc.",
    "Cass Cable TV, Inc.":"Cass Cable TV, Inc.",
    "City of Wilson GreenLight":"City of Wilson",
    "Climax Telephone Company dba CTS Telecom, Inc.":"Climax Telephone Company",
    "Cumberland Telephone Company, Inc":"CUMBERLAND TELEPHONE COMPANY",
    "Cumby Telephone Coooertive, Inc.":"Cumby Telephone Coop., Inc.",
    "Data Stream Mobile Technoligies Inc.":"Data Stream",
    "Get Wireless Inc":"Getwireless.net",
    "Hartington Telecommunications Co., Inc,":"Hartington Telecommunications Co., Inc.",
    "Hilliary Communications Consortium":"Hilliary Communications, LLC",
    "Hillsboro Telephone Company,  Inc.":"Hillsboro Telephone Company",
    "Home Telephone ILEC, LLC":"Home Telephone ILEC, LLC d/b/a Home Telecom",
    "Hood Canal Telephone Co., Inc":"Hood Canal Communications",
    "Kingdom Telecommunications, Inc.":"Kingdom Telecommunications Inc",
    "Lincolnville Communications":"Lincolnville Telephone Company",
    "Local Internet Service Company, Inc.":"Local Internet Service Company",
    "Martell Enterprise  Inc.":"Martell Enterprises, Inc.",
    "Massena Telephone Company, Inc.":"Massena Telephone Company",
    "Moundridge Telephone Company":"Moundridge Telephone Co.",
    "MyServer.org, Inc dba San Diego Broadband":"San Diego Broadband",
    "Newbreak Management, LLC":"Newbreak Communications",
    "NexGenAccess Incorporated":"NexGenAccess",
    "Northwoods Communication Technologies, LLC":"Northwoodsconnect",
    "One Point Technologies Inc":"One Point Technologies Inc.",
    "Panhandle Telecommunication Systems, Inc.":"Panhandle Telephone Cooperative, Inc.",
    "Pathwayz Communications Inc":"Pathwayz Communications, Inc.",
    "RONAN TELEPHONE COMPANY":"Ronan Telephone Co",
    "Rainbow Communications LLC":"Rainbow Telecommunications Association, Inc.",
    "SOUTHWEST ARKANSAS TELEPHONE COOPERATIVE, INC.":"Southwest Arkansas Telephone Cooperative, Inc.",
    "South Central Wireless, Inc.":"South Central Wireless Inc.",
    "Southern Montana Telephone Company (SMTC)":"Southern Montana Telephone Company",
    "Steelville Telephone Exchange Inc.":"Steelville Telephone Exchange Inc",
    "Tekstar Communications, Inc. dba Arvig":"Arvig Enterprises, Inc.",
    "Upsala Cooperative Telephone Association dba Sytek":"UPSALA COOPERATIVE TELEPHONE ASSOCIATION",
    "Valley Telephone Cooperative, Inc":"Valley Telephone Cooperative, Inc.",
    "Velocity.Net Communications, Inc.":"Velocity Communications, Inc.",
    "Woodstock Telephone Co.":"Woodstock Telephone Company",
    "ZIRKEL Wireless, LLC":"Zirkel Wireless",
    "coon valley cooperative telephone association inc.":"Coon Valley Co-op Telephone Association, Inc.",
}

bidders["bidder"] = bidders["bidder"].map(lambda x: name_map[x] if x in name_map else x)
bidders

,bidder,t+l_weight,tier,latency
0,"1stel, Inc.",20,Above Baseline,Low
1,4-Corners Consortium,20,Above Baseline,Low
2,AB Indiana LLC,20,Above Baseline,Low
3,ACT,0,Gigabit,Low
4,"AMA Communicaitons, L.L.C.",20,Above Baseline,Low
...,...,...,...,...
311,Yucca Telecom,0,Gigabit,Low
312,Zirkel Wireless,20,Above Baseline,Low
313,"Zito West Holding, LLC",0,Gigabit,Low
314,"Coon Valley Co-op Telephone Association, Inc.",0,Gigabit,Low


In [36]:
collapsed_bids = collapsed_bids.merge(bidders, how="left",left_on="winner", right_on="bidder")

In [45]:
collapsed_bids

,bidders,all_neighbors,geometry,winner,blocks,round,bidder,t+l_weight,tier,latency
0,"{'1stel, Inc.': {'neighbors': [], 'round': 9, ...","{10010202002: {('WOW', 'WideOpenWest Finance, ...","POLYGON ((-86.47038099999999 32.456581, -86.47...",Space Exploration Technologies Corp.,"[10010207001071, 10010207001073, 10010207001095]",True,Space Exploration Technologies Corp.,20.0,Above Baseline,Low
1,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010206002: {('WOW', 'WideOpenWest Finance, ...","POLYGON ((-86.59549799999999 32.381123, -86.59...",Space Exploration Technologies Corp.,"[10010208011010, 10010208011017, 1001020801102...",True,Space Exploration Technologies Corp.,20.0,Above Baseline,Low
2,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010201001: {('WOW', 'WideOpenWest Finance, ...","POLYGON ((-86.591984 32.44164, -86.591976 32.4...",Space Exploration Technologies Corp.,"[10010208012004, 10010208012010, 1001020801201...",True,Space Exploration Technologies Corp.,20.0,Above Baseline,Low
3,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010201001: {('WOW', 'WideOpenWest Finance, ...","POLYGON ((-86.645787 32.554375, -86.6457779999...",RDOF USA Consortium,"[10010208021003, 10010208021006, 1001020802101...",True,RDOF USA Consortium,0.0,Gigabit,Low
4,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010208021: {('Central Access, Inc.', 'Centr...","POLYGON ((-86.664761 32.627948, -86.6647069999...",RDOF USA Consortium,"[10010209001000, 10010209001001, 1001020900100...",True,RDOF USA Consortium,0.0,Gigabit,Low
...,...,...,...,...,...,...,...,...,...,...
61736,{'AMG Technology Investment Group LLC': {'neig...,"{160199701001: {('Safelink Internet'), ('Alway...","POLYGON ((-111.045492 43.51067099999999, -111....",Space Exploration Technologies Corp.,"[560399676003075, 560399676003087, 56039967600...",True,Space Exploration Technologies Corp.,20.0,Above Baseline,Low
61737,{'AMG Technology Investment Group LLC': {'neig...,"{560399676001: {('Rise Broadband', 'JAB Wirele...","POLYGON ((-111.039007 43.472801, -111.036624 4...",Space Exploration Technologies Corp.,"[560399677021047, 560399677021058, 56039967702...",True,Space Exploration Technologies Corp.,20.0,Above Baseline,Low
61738,{'AMG Technology Investment Group LLC': {'neig...,"{490439641011: {('Union Holding Corp.', 'Union...","POLYGON ((-111.046818 41.156019, -111.046818 4...",Space Exploration Technologies Corp.,[560419753001107],True,Space Exploration Technologies Corp.,20.0,Above Baseline,Low
61739,{'AMG Technology Investment Group LLC': {'neig...,{560039626002: {('Mountain West Technologies C...,"POLYGON ((-108.550788 44.12770099999999, -108....",Space Exploration Technologies Corp.,"[560430002001181, 560430002001513]",True,Space Exploration Technologies Corp.,20.0,Above Baseline,Low


,bidders,all_neighbors,geometry,winner,blocks,round,bidder,t+l_weight,tier,latency
0,"{'1stel, Inc.': {'neighbors': [], 'round': 9, ...","{10010202002: {('WOW', 'WideOpenWest Finance, ...","POLYGON ((-86.47038099999999 32.456581, -86.47...",Space Exploration Technologies Corp.,"[10010207001071, 10010207001073, 10010207001095]",True,Space Exploration Technologies Corp.,20.0,Above Baseline,Low
1,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010206002: {('WOW', 'WideOpenWest Finance, ...","POLYGON ((-86.59549799999999 32.381123, -86.59...",Space Exploration Technologies Corp.,"[10010208011010, 10010208011017, 1001020801102...",True,Space Exploration Technologies Corp.,20.0,Above Baseline,Low
2,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010201001: {('WOW', 'WideOpenWest Finance, ...","POLYGON ((-86.591984 32.44164, -86.591976 32.4...",Space Exploration Technologies Corp.,"[10010208012004, 10010208012010, 1001020801201...",True,Space Exploration Technologies Corp.,20.0,Above Baseline,Low
3,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010201001: {('WOW', 'WideOpenWest Finance, ...","POLYGON ((-86.645787 32.554375, -86.6457779999...",RDOF USA Consortium,"[10010208021003, 10010208021006, 1001020802101...",True,RDOF USA Consortium,0.0,Gigabit,Low
4,"{'Hughes Network Systems, LLC': {'neighbors': ...","{10010208021: {('Central Access, Inc.', 'Centr...","POLYGON ((-86.664761 32.627948, -86.6647069999...",RDOF USA Consortium,"[10010209001000, 10010209001001, 1001020900100...",True,RDOF USA Consortium,0.0,Gigabit,Low
...,...,...,...,...,...,...,...,...,...,...
61736,{'AMG Technology Investment Group LLC': {'neig...,"{160199701001: {('Safelink Internet'), ('Alway...","POLYGON ((-111.045492 43.51067099999999, -111....",Space Exploration Technologies Corp.,"[560399676003075, 560399676003087, 56039967600...",True,Space Exploration Technologies Corp.,20.0,Above Baseline,Low
61737,{'AMG Technology Investment Group LLC': {'neig...,"{560399676001: {('Rise Broadband', 'JAB Wirele...","POLYGON ((-111.039007 43.472801, -111.036624 4...",Space Exploration Technologies Corp.,"[560399677021047, 560399677021058, 56039967702...",True,Space Exploration Technologies Corp.,20.0,Above Baseline,Low
61738,{'AMG Technology Investment Group LLC': {'neig...,"{490439641011: {('Union Holding Corp.', 'Union...","POLYGON ((-111.046818 41.156019, -111.046818 4...",Space Exploration Technologies Corp.,[560419753001107],True,Space Exploration Technologies Corp.,20.0,Above Baseline,Low
61739,{'AMG Technology Investment Group LLC': {'neig...,{560039626002: {('Mountain West Technologies C...,"POLYGON ((-108.550788 44.12770099999999, -108....",Space Exploration Technologies Corp.,"[560430002001181, 560430002001513]",True,Space Exploration Technologies Corp.,20.0,Above Baseline,Low
